# Association analysis with Apriori algorithm

In [1]:
import numpy as np

In [2]:
def loadDataSet():
    return [[1, 3, 4], [2, 3, 5], [1, 2, 3, 5], [2, 5]]

In [3]:
dataSet = loadDataSet()

In [4]:
def createC1(dataSet):
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
    C1.sort()
    return list(map(frozenset, C1))

In [5]:
C1 = createC1(dataSet)
C1

[frozenset({1}),
 frozenset({2}),
 frozenset({3}),
 frozenset({4}),
 frozenset({5})]

In [6]:
def scanD(D, Ck, minSupport):
    """
    D: dataSet
    Ck: candidate items
    """
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt:
                    ssCnt[can] = 1
                else:
                    ssCnt[can] += 1
    numItems = float(len(D))
    reList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems
        if support >= minSupport:
            reList.insert(0, key)
        supportData[key] = support
    return reList, supportData

In [7]:
D = map(set, dataSet)
L1, supportData0 = scanD(dataSet, C1, 0.5)
L1

[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})]

In [8]:
def aprioriGen(Lk, k):
    """
    Lk: a list of frequent itemsets( such as {0, 1} {0, 2} {0, 3} {1, 2} {1, 3} {2, 3}...  )
    k: the size of the itemsets (such as going to merge front set into three dimension {0, 1, 2, 3}...)
    """
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i + 1, lenLk): # get random set to merge in one set
            L1 = list(Lk[i])[: k - 2] # if you want to construct three from two, compare only the k - 2 (one) element
            L2 = list(Lk[j])[: k - 2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                retList.append(Lk[i] | Lk[j]) # merge set use |(and)
    return retList

In [9]:
def apriori(dataSet, minSupport = 0.5): # generate a list of candidate itemsets
    C1 = createC1(dataSet) # use createC1() function get simple set item
    D = list(map(set, dataSet)) # list(map()) turn the dataSet into list of map
    L1, supportData = scanD(D, C1, minSupport) # scan out support less set
    L = [L1] # add L1 into a larger list
    k = 2
    while(len(L[k - 2]) > 0): # L[k - 2] express number of k-1 dimension set
        Ck = aprioriGen(L[k - 2], k) # to create k dimension set search L[k - 2](which number of k-1 dimension)
        Lk, supK = scanD(D, Ck, minSupport) # decrease support low set
        supportData.update(supK) # dict.update(dict2) 字典update()函数把字典dict2的键/值对更新到dict里
        L.append(Lk)
        k += 1
    return L, supportData

In [10]:
L, supportData = apriori(dataSet)
supportData

{frozenset({1}): 0.5,
 frozenset({3}): 0.75,
 frozenset({4}): 0.25,
 frozenset({2}): 0.75,
 frozenset({5}): 0.75,
 frozenset({1, 3}): 0.5,
 frozenset({2, 5}): 0.75,
 frozenset({3, 5}): 0.5,
 frozenset({2, 3}): 0.5,
 frozenset({1, 5}): 0.25,
 frozenset({1, 2}): 0.25,
 frozenset({2, 3, 5}): 0.5}

In [11]:
def generateRules(L, supportData, minConf = 0.7):
    """
    L: same as before dictionary [[{one-dimension}, ...], [{two-dimension}, ...], [{three-dimension}, ...]]
    supportData: support data in dictionary
    {0, 1} if there is rules as 0->1
    """
    bigRuleList = [] # save all rules with all confidence values
    for i in range(1, len(L)): # for signal item there is no rules of association
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet] # create a list of single-item sets [{0}, {1}, {2}] or [{0, 1}, {0, 2}..]
            if(i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf) # if single-item larger than 2 think to merge
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf) # for two dimension set
    return bigRuleList
    

In [12]:
def calcConf(freqSet, H, supportData, brl, minConf = 0.7):
    """
    freqSet: [{0, 1}, {0, 2}, {1, 2}....]
    H: [frozenSet(0), frozenSet(1), frozenSet(2),... ] which all of them is frozenSet
    supportData: save all support data for those itemsets
    brl: big regular set
    """
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet - conseq, '-->', conseq, 'conf', conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq) # pass rule then save 
    return prunedH

In [13]:
def rulesFromConseq(freqSet, H, supportData, brl, minConf = 0.7):
    """
    freqSet: [{0, 1, 2}, {0, 2, 3}, {1, 2, 3}.../{0, 1, 3, 4}, {1, 2, 3, 5}, .../...]
    H: [frozenSet(0), frozenSet(1), frozenSet(2),... ] which all of them is frozenSet which will on the right-hand side of a rule
    supportData: save all support data for those itemsets
    brl: big regular set
    """
    m = len(H[0])
    print(freqSet)
    print(H)
    if (len(freqSet) > (m + 1)): # 频繁项集元素数目大于单个集合的元素数
        Hmp1 = aprioriGen(H, m + 1)# 存在不同顺序、元素相同的集合，合并具有相同部分的集合
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if len(Hmp1) > 1:
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [14]:
generateRules(L, supportData, minConf = 0.7)

frozenset({5}) --> frozenset({2}) conf 1.0
frozenset({2}) --> frozenset({5}) conf 1.0
frozenset({1}) --> frozenset({3}) conf 1.0
frozenset({2, 3, 5})
[frozenset({2}), frozenset({3}), frozenset({5})]


[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]